In [111]:
from pathlib import Path
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

FILENAME = 'input.txt'


In [122]:

# Read input file
content = Path(FILENAME).read_text().splitlines()
# Create a 2D array
content = np.array([list(line) for line in content])
# replace '.' with 0 and anything that is not a number with -1
content = np.where(content == '.', 0, content)
content = np.where(np.char.isdigit(content), content, -1)
content = content.astype(int)
# add zeros to the left and right of the array to make it easier to check the edges
content = np.pad(content, ((1, 1), (1, 1)), 'constant')
content


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [123]:
# Get a list of all the adjacent char pairs by row and reshape it to a 2D array of tuples
view = sliding_window_view(content, (1, 2))
view = view.reshape(view.shape[0], view.shape[1], view.shape[2] * view.shape[3])
# Replace -1 with 0 for edge detection
edge_detect = np.where(view == -1, 0, view)
# Get the sum of each pair and the product of each pair:
# - if the sum is > 0 this means that at least one of the two cells contains the number
# - if the product is 0 this means that one of the two cells contains 0
# - if both conditions are met, this means we detected the 'edge' of a number
sums = view.sum(axis=2)
prods = view.prod(axis=2)
edges = np.logical_and(edge_detect.prod(axis=2) == 0, edge_detect.sum(axis=2) > 0)
# List the pairwise coordinates of the edges
pairs = np.argwhere(edges).reshape(-1, 2, 2)
# Add 1 to both coordinates of each first pair to compensate for the fact that it points to the cell before the edge
pairs[:, :, 1] += 1
pairs


array([[[  1,   6],
        [  1,   9]],

       [[  1,  12],
        [  1,  15]],

       [[  1,  23],
        [  1,  26]],

       ...,

       [[140,  87],
        [140,  90]],

       [[140,  95],
        [140,  97]],

       [[140, 136],
        [140, 139]]])

In [125]:
# extract the numbers from the pairs
detected = []
for pair in pairs:
    num = content[pair[0][0], pair[0][1]:pair[1][1]]
    surrounding = content[pair[0][0] - 1:pair[1][0] + 2, pair[0][1] - 1:pair[1][1] + 1]
    # if any of the cells is -1, the number is adjacent to the symbol
    if -1 in surrounding:
        # convert the array of digits to a number
        num = int(''.join(map(str, num)))
        detected.append(num)
sum(detected)


425607

In [124]:
content.shape, edge_detect.shape


((142, 142), (142, 141, 2))